# DSML WS24/25 Exercise Sheet 11 - Template Code

In [28]:
# import os
# os.environ["OMP_NUM_THREADS"] = "4" # limit numpy threads if needed.
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

Hellinger distance between power spectra:

In [29]:
def compute_and_smooth_power_spectrum(x, smoothing):
    x_ = (x - x.mean()) / x.std()
    fft_real = np.fft.rfft(x_)
    ps = np.abs(fft_real) ** 2 * 2 / len(x_)
    ps_smoothed = gaussian_filter1d(ps, smoothing)
    return ps_smoothed / ps_smoothed.sum()


def hellinger_distance(p, q):
    return np.sqrt(1 - np.sum(np.sqrt(p * q)))


def power_spectrum_error(
    X: np.ndarray, X_gen: np.ndarray, smoothing: float = 5.0
) -> float:
    """Compute the power spectrum error between two time series.

    Args:
        X (np.ndarray): Trajectory of the original system. (T x N)
        X_gen (np.ndarray): Trajectory of the generated system. (T x N)
        smoothing (float): Smoothing parameter for the power spectrum.

    Returns:
        float: Mean Hellinger distance between the power spectra of the two time series.
    """
    dists = []
    for i in range(X.shape[1]):
        ps = compute_and_smooth_power_spectrum(X[:, i], smoothing)
        ps_gen = compute_and_smooth_power_spectrum(X_gen[:, i], smoothing)
        dists.append(hellinger_distance(ps, ps_gen))
    return np.mean(dists)

## 1 Reservoir computing

1.1 Implement the ESN class

In [30]:
class ESN:
    def __init__(self, N, M, alpha, beta, sigma, rho):
        # observation space dimensionality
        self.N = N
        # reservoir size
        self.M = M
        self.alpha = alpha
        self.beta = beta
        self.sigma = sigma
        self.rho = rho

        # draw W_in from Gaussian distribution with mean 0 and variance sigma^2
        self.W_in = np.random.randn(self.M, self.N) * self.sigma

        # draw b from Gaussian distribution with mean 0 and variance beta^2
        self.b = np.random.randn(self.M) * self.beta

        # draw W randomly and renormalize to have spectral radius rho
        W = np.random.randn(self.M, self.M)
        self.W = W / np.max(np.abs(np.linalg.eigvals(W))) * self.rho

        # output weights (will be overwritten by training)
        self.W_out = None

    def forward(self, x, r):
        """Forward pass of the ESN. Implements the state equation.

        Args:
            x (np.ndarray): Input data (1D array, N)
            r (np.ndarray): Reservoir state (1D array, M)

        Returns:
            np.ndarray: Next reservoir state (1D array, M)
        """
        pass  # your code here

    def __call__(self, x, r):
        return self.forward(x, r)

    def drive(self, X):
        """Drive the ESN with input X.

        Args:
            X (np.ndarray): Input data (2D array, T x N)

        Returns:
            np.ndarray: Reservoir states (2D array, T x M)
        """
        pass  # your code here

    def train(self, X, Y, ridge_lambda, t_trans=1000):
        """Compute the output weights using ridge regression. Store the output weights in self.W_out.

        Args:
            X (np.ndarray): Input data (2D array, T x N)
            Y (np.ndarray): Target data (2D array, T x N)
            ridge_lambda (float): Ridge regression parameter
            t_trans (int, optional): Number of transient steps to discard.

        Returns:
            float: Training error
        """
        # drive the ESN with input X
        R = self.drive(X)

        # discard transient steps
        R_ = R[t_trans:, :]
        Y_ = Y[t_trans:, :]

        # compute the output weights using ridge regression -> (N x M) output weights
        self.W_out = ...  # your code here

        # compute the training error using fittet W_out
        L_RR = ...  # your code here
        return L_RR

    def generate(self, X, T_gen):
        """Generate data from the ESN.

        Args:
            X (np.ndarray): Input data (2D array, T x N)
            T_gen (int): Number of steps to generate

        Returns:
            np.ndarray: Generated data in the observation space (2D array, T_gen x N)
        """
        pass  # your code here

1.2 Train and generate data, validate the model

In [ ]:
data = np.load("lorenz_data.npy")
print(data.shape)

T_train = 10000 # use first 10000 data points for training

# split data into input (driving) and target data
X = data[:T_train, :]
Y = data[1 : T_train + 1, :]

In [ ]:
# hypers
N = 3
M = 500
alpha = 0.6
beta = 0.7
sigma = 0.3
rho = 0.75
ridge_lambda = 1e-2

In [ ]:
# initialize ESN
esn = ESN(N, M, alpha, beta, sigma, rho)

# train ESN
loss = ...  # your code here
print(loss)



# generate data using trained ESN
X_drive = X[:1000, :]
X_gen = esn.generate(X_drive, data.shape[0])

In [ ]:
# plot trajectories of respective models (plot 3d, use subplots)
# your code here

1.3 Line search for ridge regression parameter

In [33]:
# your code here